In [3]:
import spacy
import pandas as pd
import numpy as np
import yake 
import re, os, string
import pandas as pd

In [66]:
import nltk
from nltk.tokenize import sent_tokenize
text="""I am not satisfied with the boarding procedure and the flight attendant hasn't greeted me well"""
tokenized_text=sent_tokenize(text)
print(tokenized_text)

["I am not satisfied with the boarding procedure and the flight attendant hasn't greeted me well"]


In [31]:
from nltk.tokenize import word_tokenize
tokenized_word=word_tokenize(text)
print(tokenized_word)

['I', 'am', 'not', 'satisfied', 'with', 'the', 'boarding', 'procedure', 'and', 'the', 'flight', 'attendant', 'has', "n't", 'greeted', 'me', 'well']


In [4]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stop_words)

{'d', 'over', 'll', 'm', 'such', 'from', 'themselves', 're', "you've", 'i', 'be', 'on', "wasn't", 'ma', 'ain', 'my', 'needn', 'wouldn', 'have', 'but', 'after', 'own', 'there', "should've", 'yourself', 'down', 'other', 'haven', "haven't", 'am', 'because', 'hadn', "couldn't", 'being', 'their', 'again', 'with', 'below', 'yourselves', 'just', 'didn', 'some', 'it', 'doesn', 'hasn', 'all', 'myself', 'through', 'where', "shouldn't", 'out', "needn't", 'shan', 'not', "that'll", "you're", 'was', 'ourselves', 'above', 'did', 'why', 'nor', "didn't", "doesn't", "hasn't", 'then', 'when', 'his', 'them', 'further', 'so', "mustn't", 'any', 'me', 'ours', 'y', 'mightn', 'for', 'in', "it's", 'an', 'himself', 'as', 'does', "she's", "won't", 'you', 'before', 'is', 'most', "aren't", "you'd", 'no', 'this', 'had', 'to', 'wasn', 'we', 'were', 'can', 'they', 'by', 'up', 'between', 'that', 'very', "weren't", 'theirs', 'whom', 'both', 'your', "hadn't", 'shouldn', "mightn't", 'more', 'itself', 'don', 'and', 'of', '

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
def clean_text(text):
    """Doc cleaning"""
    
    # Lowering text
    text = text.lower()
    
    # Removing punctuation
    text = "".join([c for c in text if c not in PUNCTUATION])
    
    # Removing whitespace and newlines
    text = re.sub('\s+',' ',text)
    
    return text

In [7]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [8]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())

In [16]:
# Constants
PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""" 
TOP_K_KEYWORDS = 3 # top k number of keywords to retrieve in a ranked document

In [10]:
data = pd.read_csv('survey.csv').head(5)
#data.drop(['selected_text', 'sentiment'], axis=1, inplace=True)

In [11]:
data.shape

(5, 2)

In [11]:
data.dropna(subset=['text'], inplace=True)
data['text_clean'] = data['text'].apply(clean_text)
corpora = data['text_clean'].to_list()
stop_words = set(stopwords.words("english"))

In [17]:
# Initializing TF-IDF Vectorizer with stopwords
vectorizer = TfidfVectorizer(stop_words=stop_words, smooth_idf=False, use_idf=True)

# Creating vocab with our corpora
# Exlcluding first 10 docs for testing purpose
vectorizer.fit_transform(corpora)

# Storing vocab
feature_names = vectorizer.get_feature_names()

C:\Users\kastu\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
result = []
for doc in corpora[0:10]:
    df = {}
    df['full_text'] = doc
    df['top_keywords'] = get_keywords(vectorizer, feature_names, doc)
    result.append(df)
    
final = pd.DataFrame(result)
final

,full_text,top_keywords
0,the boarding procedure wasnt good didn’t like ...,"[wasnt, wait, procedure]"
1,my baggage was handled poorly will not travel ...,"[travel, poorly, handled]"
2,the flight attendent was very rude and was not...,"[rude, attendent, helpful]"
3,the flight didn’t arrive on time and was time ...,"[time, occasion, important]"
4,the flight attendant was very helpful and i li...,"[whole, liked, journey]"


# Bert

In [19]:
pip install --user keybert

Note: you may need to restart the kernel to use updated packages.


In [21]:
!pip install --user keybert[flair]
!pip install keybert[flair]
!pip install keybert[gensim]
!pip install keybert[spacy]
!pip install keybert[use]

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=822f8e8cbd6441f1c5ac8d74107c9967742f73a232e760de5037f5b93f3c3f2f
  Stored in directory: c:\users\kastu\appdata\local\pip\cache\wheels\a6\f4\e6\e40ff9021f6b3854af70fa8ea004f5ab95672817462df08fed
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-win_amd64.whl size=24150347 sha256=b4c1a

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\kastu\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\kastu\\AppData\\Local\\Temp\\pip-install-m47w8hoz\\sentencepiece_2481b68efad24055a9c3261455666c34\\setup.py'"'"'; __file__='"'"'C:\\Users\\kastu\\AppData\\Local\\Temp\\pip-install-m47w8hoz\\sentencepiece_2481b68efad24055a9c3261455666c34\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\kastu\AppData\Local\Temp\pip-wheel-upbutd7k'
       cwd: C:\Users\kastu\AppData\Local\Temp\pip-install-m47w8hoz\sentencepiece_2481b68efad24055a9c3261455666c34\
  Complete output (17 lines):
  C:\Users\kastu\anaconda3\lib\site-packages\setuptools\dist.py:757: UserWarning: Usage of dash-

  Stored in directory: c:\users\kastu\appdata\local\pip\cache\wheels\48\a5\80\fa89dc26af0f4c280b500f5529978552379c1ce8907e0a281c
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14759 sha256=5d11ed35ea59486a84501900890a740632d984084e530f6e1075c7fd4cc79b5f
  Stored in directory: c:\users\kastu\appdata\local\pip\cache\wheels\7d\37\b6\b2a79c75e898c0b8e46ff255102602d7159a10d9af0d80641a
  Created wheel for keybert: filename=keybert-0.2.0-py3-none-any.whl size=10611 sha256=a4d07d87025a4c2bfa63cd77861a882b37266a0f996bc682ae917186f66287d0
  Stored in directory: c:\users\kastu\appdata\local\pip\cache\wheels\2b\5e\cb\9bedeed618085f255420717d2960da9704821c46a9ffc1c3c3
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=47c0fac63036032562244a09d91542be5377c325258ae23ea0605ffb2795e6e1
  Stored in directory: c:\users\kastu\appdata\local\pip\cache\wheels\d1\c1\d9\7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
  Created wheel for sacre

In [59]:
from keybert import KeyBERT

doc = """
         My flight got delayed and no one has informed me the next steps



      """
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(doc)

In [60]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english')

[('flight got delayed', 0.7702),
 ('got delayed informed', 0.6279),
 ('got delayed', 0.5387),
 ('delayed informed steps', 0.5295),
 ('delayed', 0.5278)]

In [61]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english',
                              use_mmr=True, diversity=0.2)

[('flight got delayed', 0.7702),
 ('got delayed informed', 0.6279),
 ('delayed informed steps', 0.5295)]

In [62]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-roberta-large-v1')

In [63]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english')

[('flight got delayed', 0.6768),
 ('got delayed informed', 0.4217),
 ('flight got', 0.32),
 ('got delayed', 0.3187),
 ('delayed', 0.2975)]

In [35]:
!pip install --user flair

  Using cached flair-0.11.3-py3-none-any.whl (401 kB)
  Using cached konoha-4.6.5-py3-none-any.whl (20 kB)
  Using cached conllu-4.4.2-py2.py3-none-any.whl (15 kB)
  Using cached Deprecated-1.2.13-py2.py3-none-any.whl (9.6 kB)
  Using cached gdown-4.4.0-py3-none-any.whl
  Using cached mpld3-0.3-py3-none-any.whl
  Using cached pptree-3.1-py3-none-any.whl
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached Janome-0.4.2-py2.py3-none-any.whl (19.7 MB)
  Using cached langdetect-1.0.9-py3-none-any.whl
  Using cached ftfy-6.1.1-py3-none-any.whl (53 kB)
  Using cached Wikipedia_API-0.5.4-py3-none-any.whl
  Using cached sqlitedict-2.0.0-py3-none-any.whl
  Using cached bpemb-0.3.3-py3-none-any.whl (19 kB)
  Using cached more_itertools-8.13.0-py3-none-any.whl (51 kB)
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
  Using cached overrides-3.1.0-py3-none-any.whl
  Using cached importlib_metadata-3.10.1-py3-none-any.whl (14 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 4.4.0 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.


In [64]:
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings

roberta = TransformerDocumentEmbeddings('roberta-base')
kw_model = KeyBERT(model=roberta)

In [65]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=3)

[('flight got delayed', 0.9988),
 ('delayed', 0.9985),
 ('delayed informed', 0.9982)]

In [79]:
! pip install "tensorflow>=2.0.0"
! pip install --upgrade tensorflow-hub

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=94e0b7b4958c7ce47d59958bb43ce3a657559b4d72cf383ba9c465a787c56b61
  Stored in directory: c:\users\kastu\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor


In [80]:
import tensorflow_hub
embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
kw_model = KeyBERT(model=embedding_model)

In [81]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=3)

Instructions for updating:
Use tf.identity instead.


Instructions for updating:
Use tf.identity instead.


[('flight got delayed', 0.617),
 ('got delayed', 0.492),
 ('got delayed informed', 0.4698)]

In [84]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

glove_embedding = WordEmbeddings('crawl')
document_glove_embeddings = DocumentPoolEmbeddings([glove_embedding])

kw_model = KeyBERT(model=document_glove_embeddings)

2022-06-05 20:17:57,054 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M.vectors.npy not found in cache, downloading to C:\Users\kastu\AppData\Local\Temp\tmp9x07jv4i


100%|█████████████████████████████████████████████████████████████| 1200000128/1200000128 [01:01<00:00, 19450335.41B/s]

2022-06-05 20:18:59,211 copying C:\Users\kastu\AppData\Local\Temp\tmp9x07jv4i to cache at C:\Users\kastu\.flair\embeddings\en-fasttext-crawl-300d-1M.vectors.npy


2022-06-05 20:19:00,630 removing temp file C:\Users\kastu\AppData\Local\Temp\tmp9x07jv4i
2022-06-05 20:19:01,206 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M not found in cache, downloading to C:\Users\kastu\AppData\Local\Temp\tmpkfd3fjv7


100%|█████████████████████████████████████████████████████████████████| 39323680/39323680 [00:02<00:00, 13224993.62B/s]

2022-06-05 20:19:04,614 copying C:\Users\kastu\AppData\Local\Temp\tmpkfd3fjv7 to cache at C:\Users\kastu\.flair\embeddings\en-fasttext-crawl-300d-1M
2022-06-05 20:19:04,646 removing temp file C:\Users\kastu\AppData\Local\Temp\tmpkfd3fjv7


In [85]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=3)

[('flight got delayed', 0.7248),
 ('got delayed informed', 0.7067),
 ('delayed informed steps', 0.7062)]